In [1]:
import pandas as pd
import sqlite3
import math 
from tabulate import tabulate
import numpy as np

import sys
import os
# Modify path temporarily to import functions from NASEM_functions directory
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

from NASEM_functions.ration_balancing_equations import *

In [2]:
# User will type the ration into input.txt
# This function reads that file and creates a dataframe with all of the feeds and % DM 

def read_input(input):
    data = []
    animal_input = {}

    with open(input, 'r') as file:
        for line in file:
            line = line.strip()

            if line.startswith('#') or ':' not in line:
                continue

            if line.startswith('*'):
                key, value = line[1:].split(":")
                animal_input[key.strip()] = float(value.strip())
                continue

            feedstuff, per_DM = line.split(":")
            data.append([feedstuff.strip(), per_DM.strip()])

    user_input = pd.DataFrame(data, columns=["Feedstuff", "%_DM_user"])
    user_input['%_DM_user'] = user_input['%_DM_user'].astype(float)
    
    user_input['Feedstuff'] = user_input['Feedstuff'].str.strip()
    # user_input.set_index('Feedstuff', inplace=True)  # Set 'Feedstuff' as the index


    user_input['Index'] = user_input['Feedstuff']
    user_input = user_input.set_index('Index') 
    # user_input.index = user_input.index.str.strip()

    # user_input = user_input.set_index('Feedstuff')
    # user_input.set_index('Feedstuff', inplace=True)
    # user_input.index = user_input.index.str.strip()
    
    return user_input, animal_input

user_input, animal_input = read_input('input.txt')
list_of_feeds = user_input['Feedstuff'].tolist()
# list_of_feeds = user_input.index.tolist()

In [3]:
# Takes a list of the feed names and gets all their info from the feed library

def fl_get_rows(feeds_to_get):
    conn = sqlite3.connect('../../diet_database.db')
    cursor = conn.cursor()

    index_str = ', '.join([f"'{idx}'" for idx in feeds_to_get])

    query = f"SELECT * FROM NASEM_feed_library WHERE Fd_Name IN ({index_str})"

    cursor.execute(query)
    rows = cursor.fetchall()

    cursor.execute(f"PRAGMA table_info(NASEM_feed_library)")
    column_names = [column[1] for column in cursor.fetchall()]

    # Create a DataFrame from the retrieved rows with column names
    feed_data = pd.DataFrame(rows, columns=column_names)
    feed_data = feed_data.set_index('Fd_Name')
    feed_data.index = feed_data.index.str.strip()

    conn.close()

    return feed_data

feed_data = fl_get_rows(list_of_feeds)


# Get %DM to add to 100%
def set_perc_100():
    user_perc = user_input['%_DM_user'].sum()
    scaling_factor = 100 / user_perc

    user_input['%_DM_intake'] = user_input['%_DM_user'] * scaling_factor

    # Adjust so sum is exactly 100
    adjustment = 100 - user_input['%_DM_intake'].sum()
    user_input['%_DM_intake'] += adjustment / len(user_input)

    return user_input

user_input = set_perc_100()


# Predict DMI
def dmi_predicted(An_Parity_rl, Trg_MilkProd, An_BW, An_BCS, An_LactDay, Trg_MilkFatp, Trg_MilkTPp, Trg_MilkLacp):
    DMI = calculate_Dt_DMIn_Lact1(An_Parity_rl, Trg_MilkProd, An_BW, An_BCS, An_LactDay, Trg_MilkFatp, Trg_MilkTPp, Trg_MilkLacp)
    return DMI

animal_input['DMI'] = dmi_predicted(animal_input['An_Parity_rl'], animal_input['Trg_MilkProd'], animal_input['An_BW'], animal_input['An_BCS'], animal_input['An_LactDay'], animal_input['Trg_MilkFatp'], animal_input['Trg_MilkTPp'], animal_input['Trg_MilkLacp'])


# Get kg intake of each feed
def get_kg_intake(df, dict):
    DMI = dict['DMI']
    df['kg_intake'] = df['%_DM_intake']/100 * DMI

get_kg_intake(user_input, animal_input)


In [ ]:
# Get diet AA intakes OLD
# Only 10 AA: Arg, His, Ile, Leu, Lys, Met, Phe, Thr, Trp, Val

def diet_AA_intake():
    AA_list = ['Fd_Arg_CP', 'Fd_His_CP', 'Fd_Ile_CP', 'Fd_Leu_CP', 'Fd_Lys_CP', 'Fd_Met_CP', 'Fd_Phe_CP', 'Fd_Thr_CP', 'Fd_Trp_CP', 'Fd_Val_CP']
    diet_AA_intake = pd.DataFrame(columns=['Feed'] + AA_list)

    for feed in feed_data.index:
        row = {'Feed': feed}
        for AA in AA_list:
            AA_intake = (feed_data.loc[feed, AA] / 100) * (feed_data.loc[feed, 'Fd_CP'] / 100) * (animal_input['DMI'] / 100)
            row[AA] = AA_intake
        diet_AA_intake = pd.concat([diet_AA_intake, pd.DataFrame(row, index=[0])], ignore_index=True)
    
    diet_AA_intake.loc['Diet'] = diet_AA_intake.sum(numeric_only=True)
    return diet_AA_intake

diet_AA_intake = diet_AA_intake()


In [7]:
# Get diet AA intakes
# Only 10 AA: Arg, His, Ile, Leu, Lys, Met, Phe, Thr, Trp, Val
# An_305RHA_MlkTP Should be made an input value for the model 

# Do not have calculation setup to run yet so value is set here
Du_MiN_g = 328.384834667338

def AA_calculations(Du_MiN_g):
    # This function will get the intakes of AA's from the diet and then do all the calculations 
    # of values other functions will need
    # The results will be saved to a dataframe with one row for each AA and a column for each calculated value
    AA_list = ['Arg', 'His', 'Ile', 'Leu', 'Lys', 'Met', 'Phe', 'Thr', 'Trp', 'Val']
    AA_values = pd.DataFrame(index=AA_list)
    Dt_IdAARUPIn = {}

    ####################
    # Get Diet Data
    ####################
    feed_columns = ['Fd_Arg_CP', 'Fd_His_CP', 'Fd_Ile_CP', 'Fd_Leu_CP', 'Fd_Lys_CP', 'Fd_Met_CP', 'Fd_Phe_CP', 'Fd_Thr_CP', 'Fd_Trp_CP', 'Fd_Val_CP', 'Fd_dcRUP']
    df_f = pd.DataFrame(feed_data[feed_columns])
    df_f['Fd_RUPIn'] = pd.Series(user_input['Fd_RUPIn'])
   
    ####################
    # Define Variables
    ####################
    fMiTP_MiCP = 0.824			                                    # Line 1120, Fraction of MiCP that is True Protein; from Lapierre or Firkins
    SI_dcMiCP = 80				                                    # Line 1122, Digestibility coefficient for Microbial Protein (%) from NRC 2001
    K_305RHA_MlkTP = 1.0                                            # Line 2115, A scalar to adjust the slope if needed.  Assumed to be 1. MDH
    An_305RHA_MlkTP = 280                         # The default is 280 but the value for the test data is 400. This should be made an input for the model
    f_mPrt_max = 1 + K_305RHA_MlkTP * (An_305RHA_MlkTP / 280 - 1)       # Line 2116, 280kg RHA ~ 930 g mlk NP/d herd average
    Du_MiCP_g = Du_MiN_g * 6.25                                         # Line 1163
    Du_MiTP_g = fMiTP_MiCP * Du_MiCP_g                                  # Line 1166

    # AA recovery factors for recovery of each AA at maximum release in hydrolysis time over 24 h release (g true/g at 24 h)
    # From Lapierre, H., et al., 2016. Pp 205-219. in Proc. Cornell Nutrition Conference for feed manufacturers. 
    # Key roles of amino acids in cow performance and metabolism ? considerations for defining amino acid requirement. 
    # Inverted relative to that reported by Lapierre so they are true recovery factors, MDH
    RecArg = 1 / 1.061              # Line 1462-1471
    RecHis = 1 / 1.073
    RecIle = 1 / 1.12
    RecLeu = 1 / 1.065
    RecLys = 1 / 1.066
    RecMet = 1 / 1.05
    RecPhe = 1 / 1.061
    RecThr = 1 / 1.067
    RecTrp = 1 / 1.06
    RecVal = 1 / 1.102

    # Digested endogenous protein is ignored as it is a recycle of previously absorbed AA.
    # SI Digestibility of AA relative to RUP digestibility ([g dAA / g AA] / [g dRUP / g RUP])
    # All set to 1 due to lack of clear evidence for deviations.
    SIDigArgRUPf = 1
    SIDigHisRUPf = 1
    SIDigIleRUPf = 1
    SIDigLeuRUPf = 1
    SIDigLysRUPf = 1
    SIDigMetRUPf = 1
    SIDigPheRUPf = 1
    SIDigThrRUPf = 1
    SIDigTrpRUPf = 1
    SIDigValRUPf = 1

    # Microbial protein AA profile (g hydrated AA / 100 g TP) corrected for 24h hydrolysis recovery. 
    # Sok et al., 2017 JDS
    MiTPArgProf = 5.47
    MiTPHisProf = 2.21
    MiTPIleProf = 6.99
    MiTPLeuProf = 9.23
    MiTPLysProf = 9.44
    MiTPMetProf = 2.63
    MiTPPheProf = 6.30
    MiTPThrProf = 6.23
    MiTPTrpProf = 1.37
    MiTPValProf = 6.88

    # NRC derived Coefficients from Dec. 20, 2020 solutions. AIC=10,631
    # Two other sets of values are included in the R code
    
    # May be unused
    mPrt_Int_src = -97.0 
    mPrt_k_BW_src = -0.4201
    mPrt_k_DEInp_src = 10.79
    mPrt_k_DigNDF_src = -4.595
    mPrt_k_DEIn_StFA_src = 0        #DEStIn + DErOMIn + DEFAIn
    mPrt_k_DEIn_NDF_src = 0         #DENDFIn

    mPrt_k_Arg_src = 0
    mPrt_k_His_src = 1.675
    mPrt_k_Ile_src = 0.885
    mPrt_k_Leu_src = 0.466
    mPrt_k_Lys_src = 1.153	
    mPrt_k_Met_src = 1.839
    mPrt_k_Phe_src = 0
    mPrt_k_Thr_src = 0
    mPrt_k_Trp_src = 0
    mPrt_k_Val_src = 0

    # May be unused
    mPrt_k_NEAA_src = 0         #NEAA.  Phe, Thr, Trp, and Val not considered.
    mPrt_k_OthAA_src = 0.0773   #NEAA + unused EAA.  Added for NRC eqn without Arg as slightly superior.

    mPrt_k_EAA2_src = -0.00215

    
    for AA in AA_list:
        ##############################
        # Calculations on Diet Data
        ##############################
        # Fd_AAt_CP         
        df_f['Fd_{}t_CP'.format(AA)] = df_f['Fd_{}_CP'.format(AA)] / eval('Rec{}'.format(AA))

        # Fd_AARUPIn         
        df_f['Fd_{}RUPIn'.format(AA)] = df_f['Fd_{}t_CP'.format(AA)] / 100 * df_f['Fd_RUPIn'] * 1000

        # Fd_IdAARUPIn      
        df_f['Fd_Id{}RUPIn'.format(AA)] = df_f['Fd_dcRUP'] / 100 * df_f['Fd_{}RUPIn'.format(AA)] * eval('SIDig{}RUPf'.format(AA))
    
        # Dt_IdAARUPIn      
        Dt_IdAARUPIn['Dt_Id{}_RUPIn'.format(AA)] = df_f['Fd_Id{}RUPIn'.format(AA)].sum()

        ########################################
        # Calculations for Microbial Protein and Total AA Intake
        ########################################
        # Du_AAMic      
        AA_values.loc[AA, 'Du_AAMic'] = Du_MiTP_g * eval('MiTP{}Prof'.format(AA)) / 100         # Line 1573-1582

        # Du_IdAAMic        
        AA_values.loc[AA, 'Du_IdAAMic'] = AA_values.loc[AA, 'Du_AAMic'] * SI_dcMiCP / 100       # Line 1691-1700

        # Abs_AA_g
        # No infusions so Dt_IdAAIn, An_IdAA_In and Abs_AA_g are all the same value in this case
        AA_values.loc[AA, 'Abs_AA_g'] = AA_values.loc[AA, 'Du_IdAAMic'] + Dt_IdAARUPIn['Dt_Id{}_RUPIn'.format(AA)]     # Line 1703, 1714, 1757

        ########################################
        # Calculations for AA coefficients
        ########################################
        #mPrtmx_AA      CORRECT
        AA_values.loc[AA, 'mPrtmx_AA'] = -(eval('mPrt_k_{}_src'.format(AA)))**2 / (4 * mPrt_k_EAA2_src)
        
        #mPrtmx_AA2        CORRECT
        AA_values.loc[AA, 'mPrtmx_AA2'] = AA_values.loc[AA, 'mPrtmx_AA'] * f_mPrt_max                   # Line 2149-2158

        #AA_mPrtmx
        AA_values.loc[AA, 'AA_mPrtmx'] = -(eval('mPrt_k_{}_src'.format(AA))) / (2 * mPrt_k_EAA2_src)

        #mPrt_AA_0.1
        AA_values.loc[AA, 'mPrt_AA_0.1'] = AA_values.loc[AA, 'AA_mPrtmx'] * 0.1 * eval('mPrt_k_{}_src'.format(AA)) \
                                           + (AA_values.loc[AA, 'AA_mPrtmx'] * 0.1)**2 * mPrt_k_EAA2_src

        #mPrt_k_AA
        if AA_values.loc[AA, 'mPrtmx_AA2'] ** 2 - AA_values.loc[AA, 'mPrt_AA_0.1'] * AA_values.loc[AA, 'mPrtmx_AA2'] <= 0 or AA_values.loc[AA, 'AA_mPrtmx'] == 0:
        # Check for sqrt of 0 or divide by 0 errors and set value to 0 if encountered
            AA_values.loc[AA, 'mPrt_k_AA'] = 0
        else:
            AA_values.loc[AA, 'mPrt_k_AA'] = -(2 * np.sqrt(AA_values.loc[AA, 'mPrtmx_AA2'] ** 2 \
                                                - AA_values.loc[AA, 'mPrt_AA_0.1'] * AA_values.loc[AA, 'mPrtmx_AA2']) \
                                                - 2 * AA_values.loc[AA, 'mPrtmx_AA2']) \
                                                / (AA_values.loc[AA, 'AA_mPrtmx'] * 0.1)


    return df_f, AA_values, Dt_IdAARUPIn

df_f, AA_values, Dt_IdAARUPIn = AA_calculations(Du_MiN_g)


In [58]:
# Old Calculate Nutrient Intakes
# This is an older method to calculate feed intakes

def get_nutrient_intakes_OLD(df):
    for feed in df['Feedstuff']:
        
        # CP Intake
        df.loc[df['Feedstuff'] == feed, 'CP_%_diet'] = feed_data.loc[feed, 'Fd_CP'] * df.loc[df['Feedstuff'] == feed, '%_DM_intake'] / 100
        # df.loc[df['Feedstuff'] == feed, 'CP_kg/d_diet'] = (feed_data.loc[feed, 'Fd_CP'] / 100) * df.loc[df['Feedstuff'] == feed, 'kg_intake']
        df.loc[df['Feedstuff'] == feed, 'CP_kg/d_diet'] = (df.loc[df['Feedstuff'] == feed, 'CP_%_diet'] / 100) * animal_input['DMI']
        # Alternate way to calculate kg/d, shoter to write

        # RUP Intake
        df.loc[df['Feedstuff'] == feed, 'RUP_%_CP'] = feed_data.loc[feed, 'Fd_RUP_base'] * df.loc[df['Feedstuff'] == feed, '%_DM_intake'] / 100
        df.loc[df['Feedstuff'] == feed, 'RUP_%_diet'] = df.loc[df['Feedstuff'] == feed, 'RUP_%_CP'] * feed_data.loc[feed, 'Fd_CP'] / 100

        # df.loc[df['Feedstuff'] == feed, 'RUP_kg/d_diet'] = (df.loc[df['Feedstuff'] == feed, 'RUP_%_CP'] / 100) * animal_input['DMI']


# get_nutrient_intakes(user_input)

In [6]:
# Rewrite get_nutrient_intakes to use a list of feed values to calculate, add these values to user_input dataframe then delete when calculations done
# Also make functions repeat for feeds that have the same calcultion (% DM)

def get_nutrient_intakes_dev(df):
    # Remove the 'Diet' row if it exists before recalculating, otherwise, the new sum includes the old sum when calculated
    if 'Diet' in df.index:
        df = df.drop(index='Diet')

    # Define the dictionary of component names
    component_dict = {
        'Fd_CP': 'Crude Protein',
        'Fd_RUP_base': 'Rumen Undegradable Protein',
        'Fd_NDF': 'Neutral Detergent Fiber',
        'Fd_ADF': 'Acid Detergent Fiber',
        'Fd_St': 'Starch',
        'Fd_CFat': 'Crude Fat',
        'Fd_Ash': 'Ash',
        'Fd_DigNDFIn_Base': 'Digestable NDF Intake',
        'Fd_DigStIn_Base': 'Digestable Starch Intake',
        'Fd_DigrOMtIn': 'Digestable Residual Organic Matter Intake',
        'Fd_idRUPIn': 'Digested RUP',
        'Fd_DigFAIn': 'Digested Fatty Acid Intake'
    }
   
    # List any values that do not have the units % DM
    units_not_DM = ['Fd_RUP_base', 'Fd_DigNDFIn_Base', 'Fd_DigStIn_Base', 'Fd_DigrOMtIn', 'Fd_idRUPIn', 'Fd_DigFAIn'] 

    for intake, full_name in component_dict.items():
        if intake not in units_not_DM:
            df[intake] = df['Feedstuff'].map(feed_data[intake]) / 100                                # Get value from feed_data as a percentage

            df[intake + '_%_diet'] = df[intake] * df['%_DM_intake']                                  # Calculate component intake on %DM basis
            df[intake + '_kg/d_diet'] = df[intake + '_%_diet'] * animal_input['DMI'] / 100           # Calculate component kg intake 
        
        elif intake == 'Fd_RUP_base':                                                                # RUP is in % CP, so an extra conversion is needed
            df[intake] = df['Feedstuff'].map(feed_data[intake]) / 100
            df['Fd_RUP_base_%_CP'] = df[intake] * df['%_DM_intake']
            df['Fd_RUP_base_%_diet'] = df['Fd_RUP_base_%_CP'] * df['Fd_CP']
            df['Fd_RUP_base_kg/d_diet'] = df['Fd_RUP_base_%_diet'] * animal_input['DMI'] / 100
        
        elif intake == 'Fd_DigNDFIn_Base':
            df['Fd_NDFIn'] = (df['Feedstuff'].map(feed_data['Fd_NDF']) / 100) * df['kg_intake']  
            df['TT_dcFdNDF_48h'] = 12 + 0.61 * df['Feedstuff'].map(feed_data['Fd_DNDF48_NDF'])
            Use_DNDF_IV = 0
            
            if Use_DNDF_IV == 1 and df['Feedstuff'].map(feed_data['Fd_Conc']) < 100 and not np.isnan(df['TT_dcFdNDF_48h']):
                df['TT_dcFdNDF_Base'] = df['TT_dcFdNDF_48h']
            elif Use_DNDF_IV == 2 and not np.isnan(df['TT_dcFdNDF_48h']):
                df['TT_dcFdNDF_Base'] = df['TT_dcFdNDF_48h']
            else:
                def calculate_TT_dcFdNDF_Lg(Fd_NDF, Fd_Lg):
                    TT_dcFdNDF_Lg = 0.75 * (Fd_NDF - Fd_Lg) * (1 - (Fd_Lg / np.where(Fd_NDF == 0, 1e-6, Fd_NDF)) ** 0.667) / np.where(Fd_NDF == 0, 1e-6, Fd_NDF) * 100
                    return TT_dcFdNDF_Lg
                
                df['TT_dcFdNDF_Lg'] = calculate_TT_dcFdNDF_Lg(df['Feedstuff'].map(feed_data['Fd_NDF']), df['Feedstuff'].map(feed_data['Fd_Lg']))
                df['TT_dcFdNDF_Base'] = df['TT_dcFdNDF_Lg']

            df['Fd_DigNDFIn_Base'] = df['TT_dcFdNDF_Base'] / 100 * df['Fd_NDFIn']
        
        elif intake == 'Fd_DigStIn_Base':
            df['Fd_DigSt'] = df['Feedstuff'].map(feed_data['Fd_St']) * df['Feedstuff'].map(feed_data['Fd_dcSt']) / 100
            df['Fd_DigStIn_Base'] = df['Fd_DigSt'] / 100 * df['kg_intake']

        elif intake == 'Fd_DigrOMtIn':
            Fd_dcrOM = 96				                                                # Line 1005, this is a true digestbility.  There is a neg intercept of -3.43% of DM

            df['Fd_fHydr_FA'] = 1 / 1.06                                                # Line 461
            df.loc[df['Feedstuff'].map(feed_data['Fd_Category']) == "Fatty Acid Supplement", 'Fd_fHydr_FA'] = 1

            df['Fd_NPNCP'] = df['Feedstuff'].map(feed_data['Fd_CP']) * df['Feedstuff'].map(feed_data['Fd_NPN_CP']) / 100
            df['Fd_TP'] = df['Feedstuff'].map(feed_data['Fd_CP']) - df['Fd_NPNCP']

            df['Fd_NPNDM'] = df['Fd_NPNCP'] / 2.81

            df['Fd_rOM'] = 100 - df['Feedstuff'].map(feed_data['Fd_Ash']) - df['Feedstuff'].map(feed_data['Fd_NDF']) - df['Feedstuff'].map(feed_data['Fd_St']) - (df['Feedstuff'].map(feed_data['Fd_FA']) * df['Fd_fHydr_FA']) - df['Fd_TP'] - df['Fd_NPNDM'] 
            df['Fd_DigrOMt'] = Fd_dcrOM / 100 * df['Fd_rOM']
            df['Fd_DigrOMtIn'] = df['Fd_DigrOMt'] / 100 * df['kg_intake']
        
        elif intake == 'Fd_idRUPIn':
            fCPAdu = 0.064
            KpFor = 4.87        #%/h
            KpConc = 5.28	    #From Bayesian fit to Digesta Flow data with Seo Kp as priors, eqn. 26 in Hanigan et al.
            IntRUP = -0.086 	#Intercept, kg/d
            refCPIn = 3.39  	#average CPIn for the DigestaFlow dataset, kg/d.  3/21/18, MDH
            
            df['Fd_CPIn'] = df['Feedstuff'].map(feed_data['Fd_CP']) / 100 * df['kg_intake'] 
            df['Fd_CPAIn'] = df['Fd_CPIn'] * df['Feedstuff'].map(feed_data['Fd_CPARU']) / 100
            df['Fd_NPNCPIn'] = df['Fd_CPIn'] * df['Feedstuff'].map(feed_data['Fd_NPN_CP']) / 100
            df['Fd_CPBIn'] = df['Fd_CPIn'] * df['Feedstuff'].map(feed_data['Fd_CPBRU']) / 100
            df['Fd_For'] = 100 - df['Feedstuff'].map(feed_data['Fd_Conc'])
            df['Fd_RUPBIn'] = df['Fd_CPBIn'] * df['Fd_For'] / 100 * KpFor / (df['Feedstuff'].map(feed_data['Fd_KdRUP']) + KpFor) + df['Fd_CPBIn'] * df['Feedstuff'].map(feed_data['Fd_Conc']) / 100 * KpConc / (
                df['Feedstuff'].map(feed_data['Fd_KdRUP']) + KpConc)
            df['Fd_CPCIn'] = df['Fd_CPIn'] * df['Feedstuff'].map(feed_data['Fd_CPCRU']) / 100
            df['Fd_RUPIn'] = (df['Fd_CPAIn'] - df['Fd_NPNCPIn']) * fCPAdu + df['Fd_RUPBIn'] + df['Fd_CPCIn'] + IntRUP / refCPIn * df['Fd_CPIn'] 

            df['Fd_idRUPIn'] = df['Feedstuff'].map(feed_data['Fd_dcRUP']) / 100 * df['Fd_RUPIn']
        
        elif intake == 'Fd_DigFAIn':
            TT_dcFA_Base = 73
            TT_dcFat_Base = 68 

            df['TT_dcFdFA'] = df['Feedstuff'].map(feed_data['Fd_dcFA'])
            df.loc[df['Feedstuff'].map(feed_data['Fd_Category']) == "Fatty Acid Supplement", 'TT_dcFdFA'] = TT_dcFA_Base
            df.loc[df['Feedstuff'].map(feed_data['Fd_Category']) == "Fat Supplement", 'TT_dcFdFA'] = TT_dcFat_Base

            df['Fd_DigFAIn'] = (df['TT_dcFdFA'] / 100) * (df['Feedstuff'].map(feed_data['Fd_FA']) / 100) * df['kg_intake']

    # Drop the columns with feed data
    # This will need to be modified as some required columns are dropped
    # df = df.drop(columns=list(component_dict.keys()))

    # Sum component intakes
    df.loc['Diet'] = df.sum()
    df.at['Diet', 'Feedstuff'] = 'Diet'

    # Rename columns using the dictionary of component names
    df.columns = df.columns.str.replace('|'.join(component_dict.keys()), lambda x: component_dict[x.group()], regex=True)
    
    return df

user_input = get_nutrient_intakes_dev(user_input)

In [5]:
# NE calculation, to be added to function above

# The following values come from summing components of the diet and will need to be included in the calculations  
# 1. Dt_FAIn (628)
# 2. Dt_DigC160In (1274)
# 3. Dt_DigC183In (1280)

# Finished: 1. Dt_DigNDFIn_Base (594), 2. Dt_DigStIn_Base (1016), 3. Dt_DigrOMtIn (1021), 4. Dt_idRUPIn (1075), 5. Dt_NPNCPIn (609), 6. Dt_DigFAIn (1272)



def calc_NE(Dt_DigNDFIn_Base, Dt_NDFIn, An_BW, Dt_DigStIn_Base, Dt_StIn, Dt_DigrOMtIn, Dt_DMIn, An_CPIn, An_RUPIn, Dt_idRUPIn,
            Du_MiN_g, An_NDF, Dt_NPNCPIn, Dt_FAIn, Mlk_NP_g, An_BW_mature, Trg_FrmGain, An_GestDay, An_GestLength, An_LactDay, Trg_RsrvGain,
            Fet_BWbrth, An_AgeDay, An_Parity_rl, Dt_DigFAIn):
# This has been tested and works 

    ###############
    ### An_DEIn ###
    ###############
    #An_DigNDFIn#
    TT_dcNDF_Base = Dt_DigNDFIn_Base / Dt_NDFIn * 100                     # Line 1056
    if math.isnan(TT_dcNDF_Base) is True:
        TT_dcNDF_Base = 0

    An_DMIn_BW = Dt_DMIn / An_BW
    En_NDF = 4.2

    if TT_dcNDF_Base == 0:
        TT_dcNDF = 0
    else:
        TT_dcNDF = (TT_dcNDF_Base / 100 - 0.59 * (Dt_StIn / Dt_DMIn - 0.26) - 1.1 * (An_DMIn_BW - 0.035)) * 100       # Line 1060


    Dt_DigNDFIn = TT_dcNDF / 100 * Dt_NDFIn
    
    
    An_DigNDFIn = Dt_DigNDFIn + 0 * TT_dcNDF/100                                    # Line 1063, the 0 is a placeholder for InfRum_NDFIn, ask Dave about this, I think the TT_dcNDF is not needed
    An_DENDFIn = An_DigNDFIn * En_NDF                                               # Line 1353
    
    #An_DEStIn#
    En_St = 4.23                                                                   # Line 271
    TT_dcSt_Base = Dt_DigStIn_Base / Dt_StIn * 100                                 # Line 1030    
    if math.isnan(TT_dcSt_Base) is True:
        TT_dcSt_Base = 0

    if TT_dcSt_Base == 0:
        TT_dcSt = 0
    else:
        TT_dcSt = TT_dcSt_Base - (1.0 * (An_DMIn_BW - 0.035)) * 100                 # Line 1032
    An_DigStIn = Dt_StIn * TT_dcSt / 100                                            # Line 1033
    An_DEStIn = An_DigStIn * En_St                                                  # Line 1351

    #An_DErOMIn#
    En_rOM = 4.0                                                                    # Line 271
    Fe_rOMend_DMI = 3.43                                                            # Line 1005, 3.43% of DMI
    Fe_rOMend = Fe_rOMend_DMI / 100 * Dt_DMIn                               	    # Line 1007, From Tebbe et al., 2017.  Negative interecept represents endogenous rOM
    An_DigrOMaIn = Dt_DigrOMtIn - Fe_rOMend                                         # Line 1024, 1022
    An_DErOMIn = An_DigrOMaIn * En_rOM                                              # Line 1352

    #An_DETPIn#
    SI_dcMiCP = 80			                                                    	# Line 1123, Digestibility coefficient for Microbial Protein (%) from NRC 2001 
    En_CP = 5.65                                                                    # Line 266
    dcNPNCP = 100	                                                                # Line 1092, urea and ammonium salt digestibility
    En_NPNCP = 0.89                                                                 # Line 270
    An_idRUPIn = Dt_idRUPIn                                       # Line 1099
    Fe_RUP = An_RUPIn - An_idRUPIn                                                  # Line 1198   
    Du_MiCP_g = Du_MiN_g * 6.25                                                     # Line 1164
    Du_MiCP = Du_MiCP_g / 1000                                                      # Line 1166
    Du_idMiCP_g = SI_dcMiCP / 100 * Du_MiCP_g
    Du_idMiCP = Du_idMiCP_g / 1000
    Fe_RumMiCP = Du_MiCP - Du_idMiCP                                                # Line 1196
    Fe_CPend_g = (12 + 0.12 * An_NDF) * Dt_DMIn            # line 1187, g/d, endogen secretions plus urea capture in microbies in rumen and LI
    Fe_CPend = Fe_CPend_g / 1000                                                    # Line 1190
    Fe_CP = Fe_RUP + Fe_RumMiCP + Fe_CPend          # Line 1202, Double counting portion of RumMiCP derived from End CP. Needs to be fixed. MDH
    An_DigCPaIn = An_CPIn - Fe_CP		            # Line 1222, apparent total tract
    An_DECPIn = An_DigCPaIn * En_CP
    An_DENPNCPIn = Dt_NPNCPIn * dcNPNCP / 100 * En_NPNCP                                                          # Line 1355, 1348
    An_DETPIn = An_DECPIn - An_DENPNCPIn / En_NPNCP * En_CP                       # Line 1356, Caution! DigTPaIn not clean so subtracted DE for CP equiv of NPN to correct. Not a true DE_TP.

    #An_DEFAIn#
    En_FA = 9.4                                                                                         # Line 265
    An_DigFAIn = Dt_DigFAIn                                                                             # Line 1309
    An_DEFAIn = An_DigFAIn * En_FA

    An_DEIn = An_DENDFIn + An_DEStIn + An_DErOMIn + An_DETPIn + An_DENPNCPIn + An_DEFAIn  # Line 1367

    #######################
    ### An_GasEOut_Lact ###
    #######################
    An_DigNDF = An_DigNDFIn / Dt_DMIn * 100
    An_GasEOut_Lact = 0.294 * Dt_DMIn - 0.347 * Dt_FAIn / Dt_DMIn * 100 + 0.0409 * An_DigNDF

    ################
    ### Ur_DEout ###
    ################
    Scrf_CP_g = 0.20 * An_BW**0.60                                                                      # Line 1965
    Mlk_CP_g = Mlk_NP_g / 0.95                                          # Line 2213
    CPGain_FrmGain = 0.201 - 0.081 * An_BW / An_BW_mature
    Body_NP_CP = 0.86                                                      # Line 1964
    Frm_Gain = Trg_FrmGain
    An_GutFill_BW = 0.18                                                   # Line 2400 and 2411
    Frm_Gain_empty = Frm_Gain * (1 - An_GutFill_BW)
    NPGain_FrmGain = CPGain_FrmGain * Body_NP_CP                           # Line 2460
    Frm_NPgain = NPGain_FrmGain * Frm_Gain_empty                           # Line 2461
    CPGain_RsrvGain = 0.068                                           # Line 2466
    NPGain_RsrvGain = CPGain_RsrvGain * Body_NP_CP                    # Line 2467
    Rsrv_Gain_empty = Trg_RsrvGain                                    # Line 2435 and 2441
    Rsrv_NPgain = NPGain_RsrvGain * Rsrv_Gain_empty                    # Line 2468
    Body_NPgain = Frm_NPgain + Rsrv_NPgain
    Body_CPgain = Body_NPgain / Body_NP_CP                                  # Line 2475
    Body_CPgain_g = Body_CPgain * 1000                                      # Line 2477

    #Gest_CPuse_g#
    GrUter_Ksyn = 2.43e-2                                         # Line 2302
    GrUter_KsynDecay = 2.45e-5                                    # Line 2303
    UterWt_FetBWbrth = 0.2311                                     # Line 2296
    Uter_Wtpart = Fet_BWbrth * UterWt_FetBWbrth                   # Line 2311
    Uter_Ksyn = 2.42e-2                                           # Line 2306
    Uter_KsynDecay = 3.53e-5                                      # Line 2307
    Uter_Kdeg = 0.20                                              # Line 2308
    Uter_Wt = 0.204                                               # Line 2312-2318
    
    if An_AgeDay < 240:
      Uter_Wt = 0
    
    if An_GestDay > 0 and An_GestDay <= An_GestLength:
      Uter_Wt = Uter_Wtpart * math.exp(-(Uter_Ksyn-Uter_KsynDecay*An_GestDay)*(An_GestLength-An_GestDay))

    if An_GestDay <= 0 and An_LactDay > 0 and An_LactDay < 100:
      Uter_Wt = ((Uter_Wtpart-0.204)* math.exp(-Uter_Kdeg*An_LactDay))+0.204

    if An_Parity_rl > 0 and Uter_Wt < 0.204:
      Uter_Wt = 0.204
    
    GrUterWt_FetBWbrth = 1.816                                    # Line 2295
    GrUter_Wtpart = Fet_BWbrth * GrUterWt_FetBWbrth               # Line 2322
    GrUter_Wt = Uter_Wt                                           # Line 2323-2327   

    if An_GestDay > 0 and An_GestDay <= An_GestLength:
      GrUter_Wt = GrUter_Wtpart * math.exp(-(GrUter_Ksyn-GrUter_KsynDecay*An_GestDay)*(An_GestLength-An_GestDay))

    if GrUter_Wt < Uter_Wt:
      GrUter_Wt = Uter_Wt
    
    Uter_BWgain = 0  #Open and nonregressing animal

    if An_GestDay > 0 and An_GestDay <= An_GestLength:
      Uter_BWgain = (Uter_Ksyn - Uter_KsynDecay * An_GestDay) * Uter_Wt

    if An_GestDay <= 0 and An_LactDay > 0 and An_LactDay < 100:
      Uter_BWgain = -Uter_Kdeg*Uter_Wt
    
    GrUter_BWgain = 0                                              # Line 2341-2345

    if An_GestDay > 0 and An_GestDay <= An_GestLength:
      GrUter_BWgain = (GrUter_Ksyn-GrUter_KsynDecay*An_GestDay)*GrUter_Wt

    if An_GestDay <= 0 and An_LactDay > 0 and An_LactDay < 100:
      GrUter_BWgain = Uter_BWgain
 
    CP_GrUtWt = 0.123                                               # Line 2298, kg CP/kg fresh Gr Uterus weight
    Gest_NPother_g = 0                                              # Line 2353, Net protein gain in other maternal tissues during late gestation: mammary, intestine, liver, and blood. This should be replaced with a growth funncton such as Dijkstra's mammary growth equation. MDH.                                                              
    Gest_NCPgain_g = GrUter_BWgain * CP_GrUtWt * 1000
    Gest_NPgain_g = Gest_NCPgain_g * Body_NP_CP
    Gest_NPuse_g = Gest_NPgain_g + Gest_NPother_g                             # Line 2366
    Gest_CPuse_g = Gest_NPuse_g / Body_NP_CP                                  # Line 2367
    Ur_Nout_g = (An_CPIn * 1000 - Fe_CP * 1000 - Scrf_CP_g - Fe_CPend_g - Mlk_CP_g - Body_CPgain_g - Gest_CPuse_g) / 6.25     # Line 2742
    Ur_DEout = 0.0143 * Ur_Nout_g                               # Line 2748

    An_MEIn = An_DEIn - An_GasEOut_Lact - Ur_DEout
    An_NE_In = An_MEIn * 0.66                                  # Line 2762
    An_NE = An_NE_In / Dt_DMIn                                 # Line 2763

    return An_NE


In [ ]:
# Use this to compare feeds with NASEM 8 calculations
print(user_input['Fd_RUP_base_%_diet'].sum())
print(user_input['Fd_RUP_base_kg/d_diet'].sum())
print(user_input.columns.values)

In [40]:
def display_diet_values(df):
    components = ['Crude Protein', 'Rumen Undegradable Protein', 'Neutral Detergent Fiber', 'Acid Detergent Fiber', 'Starch', 'Crude Fat', 'Ash']
    rows = []

    for component in components:
        percent_diet = round(df.loc['Diet', component + '_%_diet'].values[0], 2)
        kg_diet = round(df.loc['Diet', component + '_kg/d_diet'].values[0], 2)
        rows.append([component, percent_diet, kg_diet])

    headers = ['Component', '% DM', 'kg/d']

    table = tabulate(rows, headers=headers, tablefmt='fancy_grid', stralign="center")

    print(table)

# Call the function with your 'user_input' dataframe
display_diet_values(user_input)


╒════════════════════════════╤════════╤════════╕
│         Component          │   % DM │   kg/d │
╞════════════════════════════╪════════╪════════╡
│       Crude Protein        │  21.07 │   5.17 │
├────────────────────────────┼────────┼────────┤
│ Rumen Undegradable Protein │   6.65 │   1.63 │
├────────────────────────────┼────────┼────────┤
│  Neutral Detergent Fiber   │  33.04 │   8.1  │
├────────────────────────────┼────────┼────────┤
│    Acid Detergent Fiber    │  22.95 │   5.63 │
├────────────────────────────┼────────┼────────┤
│           Starch           │  20.25 │   4.97 │
├────────────────────────────┼────────┼────────┤
│         Crude Fat          │   3    │   0.74 │
├────────────────────────────┼────────┼────────┤
│            Ash             │   7.19 │   1.76 │
╘════════════════════════════╧════════╧════════╛


In [10]:
# This is a way to display multiple tables beside eachother

# data1 = {'Col1': [1, 2, 3], 'Col2': ['A', 'B', 'C']}
# data2 = {'Col3': [4, 5, 6], 'Col4': ['D', 'E', 'F']}
data1 = {'Col1': [1, 2, 3, 4], 'Col2': ['A', 'B', 'C', 'D']}
data2 = {'Col3': [5, 6], 'Col4': ['E', 'F']}

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# Convert dataframes to tabulate tables
table1 = tabulate(df1, headers='keys', tablefmt='fancy_grid')
table2 = tabulate(df2, headers='keys', tablefmt='fancy_grid')

# Split table strings into rows
table1_rows = table1.split('\n')
table2_rows = table2.split('\n')

# Find the maximum number of rows between the two tables
max_rows = max(len(table1_rows), len(table2_rows))

# Add empty rows if needed to make both tables have the same number of rows
table1_rows += [''] * (max_rows - len(table1_rows))
table2_rows += [''] * (max_rows - len(table2_rows))

# Combine the rows from each table side by side
combined_rows = [f'{row1}    {row2}' for row1, row2 in zip(table1_rows, table2_rows)]

# Print the combined table
print('\n'.join(combined_rows))

╒════╤════════╤════════╕    ╒════╤════════╤════════╕
│    │   Col1 │ Col2   │    │    │   Col3 │ Col4   │
╞════╪════════╪════════╡    ╞════╪════════╪════════╡
│  0 │      1 │ A      │    │  0 │      5 │ E      │
├────┼────────┼────────┤    ├────┼────────┼────────┤
│  1 │      2 │ B      │    │  1 │      6 │ F      │
├────┼────────┼────────┤    ╘════╧════════╧════════╛
│  2 │      3 │ C      │    
├────┼────────┼────────┤    
│  3 │      4 │ D      │    
╘════╧════════╧════════╛    
